In [ ]:
import pandas as pd
import numpy as np
import tqdm
from rdkit import Chem
from ase import Atoms
from ase.io import read, write
from ase.calculators.emt import EMT
import pickle

from mace.calculators import mace_mp, mace_off, mace_anicc, MACECalculator

params = Chem.SmilesParserParams()
params.removeHs = False

/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat,

In [8]:
# Folder for the experiment (rdb7 or rgd1) in this folder there needs to be a train/val/test csv file and a train/val/test xyz file.
# ATTENTION: The xyz files need to be in the same order as the csv files.
path = "data/rdb7/"   
dataset = "rdb7"     

In [4]:
#### Calculate Descriptors using the MACE model ####
####################################################

macemp = mace_mp() # This one can be changed to mace_off or mace_anicc

for split in ["train", "test", "val"]:

    df = pd.read_csv(f"{path}/{split}.csv")
    xyz_data = read(f"{path}/{split}.xyz", ":")

    all_desc = []
    xyz_index = 0

    for index, row in tqdm.tqdm(df.iterrows()):

        smiles = df.loc[index]["smiles"].split(">>")[0]

        params = Chem.SmilesParserParams()
        params.removeHs = False
        mol = Chem.MolFromSmiles(smiles, params)

        atom_map_numbers = np.array([a.GetAtomMapNum() for a in mol.GetAtoms()])

        atom_symbols = xyz_data[xyz_index].get_chemical_symbols()
        sorted_atom_symbols = [
            atom_symbols[i - 1] for i in atom_map_numbers
        ]  # Sort atom symbols according to atom mapping numbers

        positions = xyz_data[xyz_index].get_positions()
        sorted_positions = [
            positions[i - 1] for i in atom_map_numbers
        ]  # Sort positions according to atom mapping numbers

        ### Positions for TS #####
        xyz_index += 1
        positions_ts = xyz_data[xyz_index].get_positions()
        sorted_positions_ts = [positions_ts[i - 1] for i in atom_map_numbers]

        ### Positions for Product #####
        xyz_index += 1
        positions_prod = xyz_data[xyz_index].get_positions()
        sorted_positions_prod = [positions_prod[i - 1] for i in atom_map_numbers]
        xyz_index += 1

        molecule = Atoms(positions=sorted_positions, symbols=sorted_atom_symbols)
        molecule_ts = Atoms(positions=sorted_positions_ts, symbols=sorted_atom_symbols)
        molecule_prod = Atoms(
            positions=sorted_positions_prod, symbols=sorted_atom_symbols
        )

        desc = macemp.get_descriptors(molecule)
        desc_ts = macemp.get_descriptors(molecule_ts)
        desc_prod = macemp.get_descriptors(molecule_prod)

        # all_desc.append(np.hstack([desc, desc_ts, desc_prod])) # Uncomment if you want to save the descriptors for the product and reactant as well
        all_desc.append(desc_ts)

    np.savez_compressed(f"{path}/{split}_mace_mp_ts.npz", *all_desc)

Using Materials Project MACE for MACECalculator with /home/johannes.karwounopoulos/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.lo

Default dtype float32 does not match model dtype float64, converting models to float32.


10733it [11:00, 16.25it/s]
597it [00:38, 15.64it/s]
596it [00:37, 15.96it/s]


In [ ]:
### Create descriptor file for the flow matching model ###
###########################################################

split = "test"

coord = f"coordinates_flowMatching.pkl"

with open(path+"/"+coord, "rb") as f:
    coordinate_file = pickle.load(f)

smiles_file = pd.read_csv(f"data/{dataset}/{split}.csv")

# ## Flow Matching model
assert len(coordinate_file) == len(smiles_file)
assert coordinate_file[1].smiles == smiles_file["smiles"][1]

macemp = mace_mp()
all_desc = []

for index, row in smiles_file.iterrows():
    rtsp = 0
    smiles_data = smiles_file.iloc[index].smiles.split(">>")[0]
    smiles = coordinate_file[index].smiles.split(">>")[0]

    mol = Chem.MolFromSmiles(smiles, params)

    atom_map_num = np.array([a.GetAtomMapNum() for a in mol.GetAtoms()])
    atom_symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
    atom_symbols_mapped = [atom_symbols[i] for i in np.argsort(atom_map_num)]

    assert smiles_data == smiles

    positions = coordinate_file[index].pos_gen[-1]

    symbols = []
    new_positions = []
    for atom in mol.GetAtoms():
        atom_num = atom.GetAtomMapNum()
        symbols.append(atom_symbols_mapped[atom_num - 1])
        new_positions.append(positions[atom_num - 1])

    molecule_ts = Atoms(symbols=symbols, positions=new_positions)
    desc_ts = macemp.get_descriptors(molecule_ts)

    all_desc.append(desc_ts)

np.savez_compressed(
    f"data/{dataset}/{split}_mace_mp_ts_flowMatching.npz", *all_desc
)


/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.loa

Using Materials Project MACE for MACECalculator with /home/johannes.karwounopoulos/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.lo

In [ ]:
## Create descriptor file for the diffusion model ##
######################################################

split = "test"
coord = f"coordinates_diffusion.pkl" 

with open(path+"/"+coord, "rb") as f:
    coordinate_file = pickle.load(f)

smiles_file = pd.read_csv(f"data/{dataset}/{split}.csv")

assert len(coordinate_file[0]) == len(smiles_file)
assert coordinate_file[0][1].smiles == smiles_file["smiles"][1]

macemp = mace_mp()
all_desc = []


for index, row in smiles_file.iterrows():
    rtsp = 0
    smiles_data = smiles_file.iloc[index].smiles.split(">>")[0]
    smiles = coordinate_file[rtsp][index].smiles.split(">>")[0]

    mol = Chem.MolFromSmiles(smiles, params)

    atom_map_num = np.array([a.GetAtomMapNum() for a in mol.GetAtoms()])
    atom_symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
    atom_symbols_mapped = [atom_symbols[i] for i in np.argsort(atom_map_num)]

    assert smiles_data == smiles

    positions = coordinate_file[1][index].pos_gen

    symbols = []
    new_positions = []
    for atom in mol.GetAtoms():
        atom_num = atom.GetAtomMapNum()
        symbols.append(atom_symbols_mapped[atom_num - 1])
        new_positions.append(positions[atom_num - 1])

    molecule_ts = Atoms(symbols=symbols, positions=new_positions)
    desc_ts = macemp.get_descriptors(molecule_ts)

    all_desc.append(desc_ts)

np.savez_compressed(
    f"data/{dataset}/{split}_mace_mp_ts_diffusion.npz", *all_desc
)


/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.loa

Using Materials Project MACE for MACECalculator with /home/johannes.karwounopoulos/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/johannes.karwounopoulos/miniconda3/envs/deeprxn_paper/lib/python3.12/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.lo